In [3]:
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
import numpy as np


In [2]:
# Carregando o Dataset
df = pd.read_pickle('../storage/pickle/dataframe_amostra_no_dummies0.15.pkl')
df = df.drop_duplicates()
# Tratando dado de idade
df["idade"] = pd.to_numeric(df["idade"])

In [3]:
# Conecta ao banco de dados
conn = psycopg2.connect(
    host="localhost",
    database="previsao-patologias",
    user="postgres",
    password="test"
)

cursor = conn.cursor()
query = "SELECT texto FROM tb_questoes WHERE tipo_resposta = 'binaria';"
cursor.execute(query)

# Pega resultado da query e transforma em lista para realizar iteração
tuplas_resultados = cursor.fetchall()
questoes = [item for tupla in tuplas_resultados for item in tupla]

In [4]:
questoes_binarias = []
for c in questoes:
    nome_coluna_tratado = c.replace(" ", "_").replace(".", "").replace("?", "").lower()
    questoes_binarias.append(nome_coluna_tratado)
    
colunas_dummies = [col for col in df.columns if col not in ['idade', 'patologia', 'severidade'] + questoes_binarias]

df_dummies = pd.get_dummies(df, columns=colunas_dummies, dtype=int)

Separando variáveis dependentes e de testes e criando conjunto de treino e testes

In [5]:
variaveis_dependentes = df_dummies.drop(columns=['patologia', 'severidade'])
variavel_target = df_dummies['patologia']

In [6]:
# Splitando conjuntos de treino e testes
X_train, X_test, y_train, y_test = train_test_split(variaveis_dependentes, variavel_target, test_size=0.2, random_state=42)

In [7]:
# Criar um modelo nulo (classificação majoritária)
dummy_model = DummyClassifier(strategy='uniform')

# Treinar o modelo nulo no conjunto de treinamento
dummy_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_null = dummy_model.predict(X_test)

# Avaliar o desempenho do modelo nulo
accuracy_null = accuracy_score(y_test, y_pred_null)
print(f'Acurácia do modelo nulo: {accuracy_null}')

Acurácia do modelo nulo: 0.019292355371900827


In [8]:
# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred_null)
classification_report_result = classification_report(y_test, y_pred_null)
matriz_confusao = confusion_matrix(y_test, y_pred_null, labels=dummy_model.classes_)
f1 = f1_score(y_test, y_pred_null, average='macro')

# Calcular sensibilidade (recall) e especificidade para cada classe
sensibilidade = np.diag(matriz_confusao) / np.sum(matriz_confusao, axis=1)

# Calcular sensibilidade e especificidade médias (macro)
sensibilidade_media = np.mean(sensibilidade)

# Exibir métricas de desempenho
print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:\n', classification_report_result)
print(f"F1-Score: {f1}")
print(f'Sensibilidade (Recall) Média: {sensibilidade_media}')

Acurácia: 0.019292355371900827
Relatório de Classificação:
                                           precision    recall  f1-score   support

     acute copd exacerbation / infection       0.02      0.02      0.02       660
                acute dystonic reactions       0.02      0.02      0.02      1017
                        acute laryngitis       0.03      0.03      0.03       905
                      acute otitis media       0.03      0.02      0.02       978
                   acute pulmonary edema       0.02      0.02      0.02       699
                    acute rhinosinusitis       0.01      0.01      0.01       487
                      allergic sinusitis       0.02      0.02      0.02       879
                             anaphylaxis       0.04      0.03      0.03      1056
                                  anemia       0.05      0.02      0.03      1939
                     atrial fibrillation       0.02      0.01      0.02       807
                               boerha

In [8]:
# Carregando o Dataset
df = pd.read_pickle('../storage/pickle/dataframe_smote0.10.pkl')
df = df.drop_duplicates()
# Tratando dado de idade
df["idade"] = pd.to_numeric(df["idade"])

In [9]:
variaveis_dependentes = df.drop(columns=['patologia', 'severidade'])
variavel_target = df['patologia']

In [10]:
# Splitando conjuntos de treino e testes
X_train, X_test, y_train, y_test = train_test_split(variaveis_dependentes, variavel_target, test_size=0.2, random_state=42)

In [11]:
# Criar um modelo nulo (classificação majoritária)
dummy_model = DummyClassifier(strategy='uniform')

# Treinar o modelo nulo no conjunto de treinamento
dummy_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_null = dummy_model.predict(X_test)

# Avaliar o desempenho do modelo nulo
accuracy_null = accuracy_score(y_test, y_pred_null)
print(f'Acurácia do modelo nulo: {accuracy_null}')

Acurácia do modelo nulo: 0.020135917442738484


In [13]:
# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred_null)
classification_report_result = classification_report(y_test, y_pred_null)
matriz_confusao = confusion_matrix(y_test, y_pred_null, labels=dummy_model.classes_)
f1 = f1_score(y_test, y_pred_null, average='macro')

# Calcular sensibilidade (recall) e especificidade para cada classe
sensibilidade = np.diag(matriz_confusao) / np.sum(matriz_confusao, axis=1)

# Calcular sensibilidade e especificidade médias (macro)
sensibilidade_media = np.mean(sensibilidade)

# Exibir métricas de desempenho
print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:\n', classification_report_result)
print(f"F1-Score: {f1}")
# print(f'Sensibilidade (Recall) Média: {sensibilidade_media}')

Acurácia: 0.020135917442738484
Relatório de Classificação:
                                           precision    recall  f1-score   support

     acute copd exacerbation / infection       0.03      0.03      0.03       803
                acute dystonic reactions       0.02      0.02      0.02       781
                        acute laryngitis       0.02      0.02      0.02       829
                      acute otitis media       0.01      0.01      0.01       850
                   acute pulmonary edema       0.02      0.02      0.02       796
                    acute rhinosinusitis       0.02      0.02      0.02       820
                      allergic sinusitis       0.02      0.02      0.02       789
                             anaphylaxis       0.02      0.02      0.02       851
                                  anemia       0.02      0.03      0.02       775
                     atrial fibrillation       0.02      0.02      0.02       797
                               boerha